In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora
from gensim.models import TfidfModel
import pyLDAvis
import pyLDAvis.gensim
import pandas as pd

In [4]:
import nltk

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
nltk.download('words')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [2]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 42.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [ ]:
#Todo: Delete three letter words
# List of removal: high value words
# Remove non english words - DONE
# Also create python file where instead of visualisation it is just dataframe creations - DONE

In [20]:
class topic_modeling:
    def __init__(self, facts):
        self.facts = facts

    def organise(self):
        facts_words = []
        for fact in self.facts:
            split_fact = fact.split()
            facts_words.append(split_fact)
        return facts_words

    def remove_lang_words(self,facts_words, english_words):
    
      # Filtering out non-english words and three letter words
      filtered = [word for word in facts_words if word in english_words and len(word)>3]
      return filtered

    def tf_idf(self, id2word, texts, corpus, tfidf):
        # initialise a dictionary
        #id2word = corpora.Dictionary(data_bigrams_trigrams)

        #texts = data_bigrams_trigrams

        #corpus = [id2word.doc2bow(text) for text in texts]
        # print (corpus[0][0:20])

        #tfidf = TfidfModel(corpus, id2word=id2word)

        low_value = 0.03
        high_value = 0.6
        words = []
        words_missing_in_tfidf = []
        for i in range(0, len(corpus)):
            bow = corpus[i]
            value_words = []  # reinitialize to be safe. You can skip this.
            tfidf_ids = [id for id, value in tfidf[bow]]
            bow_ids = [id for id, value in bow]
            value_words = [id for id, value in tfidf[bow] if value < low_value or value > high_value]
            drops = value_words + words_missing_in_tfidf 
            
            for item in drops:
                words.append(id2word[item])
            words_missing_in_tfidf = [id for id in bow_ids if
                                      id not in tfidf_ids]  # The words with tf-idf socre 0 will be missing

            new_bow = [b for b in bow if b[0] not in value_words and b[0] not in words_missing_in_tfidf]
            corpus[i] = new_bow

        return corpus, id2word

    def perform_lda(self, corpus, id2word):
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                    id2word=id2word,
                                                    num_topics=30,
                                                    random_state=100,
                                                    update_every=1,
                                                    chunksize=100,
                                                    passes=10,
                                                    alpha="auto")

        return lda_model


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
from google.colab import files
import io
uploaded = files.upload()
violation_tm = pd.read_csv(io.BytesIO(uploaded['violation_facts_processed.csv']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving violation_facts_processed.csv to violation_facts_processed.csv


In [9]:
violation_tm.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,Facts,Cluster,x0,x1
0,0,life warsaw represented gniady practising wars...,NaN,NaN,NaN
1,1,born serving life prison pärnu prison requeste...,NaN,NaN,NaN
2,2,born life glina background indicted sisak coun...,NaN,NaN,NaN
3,3,born detained represented albuga çalıkuşu prac...,NaN,NaN,NaN
4,4,issue born life weinheim campaign abortion dis...,NaN,NaN,NaN


In [23]:
facts = violation_tm['Facts'].tolist()
tm = topic_modeling(facts)
facts_words = tm.organise()
print(len(facts_words))

207


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
print(facts_words[0])

['life', 'warsaw', 'represented', 'gniady', 'practising', 'warsaw', 'polish', 'represented', 'agent', 'sobczak', 'ministry', 'foreign', 'affair', 'summarised', 'lodged', 'behalf', 'client', 'company', 'formal', 'notification', 'crime', 'contained', 'following', 'statement', 'data', 'exists', 'sufficiently', 'justify', 'suspicion', 'full', 'name', 'service', 'committed', 'bribe', 'taking', 'mean', 'accepting', 'bribe', 'unknown', 'probably', 'employee', 'company', 'mentioned', 'name', 'say', 'code', 'document', 'explained', 'question', 'committed', 'connection', 'investigation', 'forging', 'contract', 'detriment', 'client', 'ongoing', 'time', 'particular', 'alleged', 'request', 'filed', 'course', 'abovementioned', 'investigation', 'examined', 'promptly', 'shortcoming', 'occurred', 'overall', 'investigation', 'question', 'prematurely', 'unjustifiably', 'discontinued', 'ground', 'insufficient', 'justify', 'suspicion', 'crime', 'committed', 'expert', 'report', 'obtained', 'view', 'explanat

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
final_data = []
english_words = set(nltk.corpus.words.words())
for i in range(0, len(facts_words)):
  facts_words_filtered = tm.remove_lang_words(facts_words[i], english_words)
  final_data.append(facts_words_filtered)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
print(final_data[0])

['life', 'warsaw', 'warsaw', 'polish', 'agent', 'ministry', 'foreign', 'affair', 'lodged', 'behalf', 'client', 'company', 'formal', 'notification', 'crime', 'following', 'statement', 'data', 'sufficiently', 'justify', 'suspicion', 'full', 'name', 'service', 'bribe', 'taking', 'mean', 'bribe', 'unknown', 'probably', 'employee', 'company', 'name', 'code', 'document', 'question', 'connection', 'investigation', 'forging', 'contract', 'detriment', 'client', 'ongoing', 'time', 'particular', 'request', 'course', 'investigation', 'promptly', 'shortcoming', 'overall', 'investigation', 'question', 'prematurely', 'unjustifiably', 'ground', 'insufficient', 'justify', 'suspicion', 'crime', 'expert', 'report', 'view', 'explanation', 'irrational', 'discontinue', 'investigation', 'charge', 'investigation', 'question', 'notification', 'contain', 'detail', 'regarding', 'allegedly', 'bribe', 'name', 'company', 'supposedly', 'employed', 'warsaw', 'district', 'open', 'inquiry', 'reasoning', 'stated', 'sing

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
bigram_phrases = gensim.models.Phrases(final_data, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[final_data], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(final_data)
data_bigrams_trigrams = make_trigrams(data_bigrams)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder, QuadgramAssocMeasures, QuadgramCollocationFinder
from nltk import word_tokenize

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [50]:
def collocations(facts):
        # collocation: pair or group of words that appear together in a specific language
        # finding collections requires first calculating the frequencies of words and their appearance in the context of other words
        # n-grams are contiguous sequences of N items from a given sample of text (bigram: pair of consecutive words in a sentence or document)
        # each ngram of words may then be scored according to some association measure, in order to determine the relative likelihood of each ngram being a collection

        # CollectionFinder: scores a ngram given appropriate frequency

        # tokenize
        text = ' '.join(facts)
        tokens = word_tokenize(text)

        bigrams = BigramAssocMeasures()
        trigrams = TrigramAssocMeasures()
        quadgrams = QuadgramAssocMeasures()

        bigramFinder = BigramCollocationFinder.from_words(tokens)
        trigramFinder = TrigramCollocationFinder.from_words(tokens)
        quadgramFinder = QuadgramCollocationFinder.from_words(tokens)

        return bigrams, trigrams, quadgrams, bigramFinder, trigramFinder, quadgramFinder

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [63]:
def find_bigrams(facts):
    text = ' '.join(facts)
    tokens = word_tokenize(text)
    bigrams = list(nltk.bigrams(tokens))
    return bigrams

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [51]:
bi_finder_collection = []
for data in final_data:
  bigrams, trigrams, quadgrams, bigramFinder, trigramFinder, quadgramFinder = collocations(data)
  bi_finder_collection.append(bigramFinder)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [62]:
ten_best = bigramFinder.nbest(bigrams.pmi, 10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [57]:
ten_best[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


('accurate', 'reliable')

In [64]:
bigram_list = []
for data in final_data:
  bigrams = find_bigrams(data)
  bigram_list.append(bigrams)

print(bigram_list[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('life', 'warsaw'), ('warsaw', 'warsaw'), ('warsaw', 'polish'), ('polish', 'agent'), ('agent', 'ministry'), ('ministry', 'foreign'), ('foreign', 'affair'), ('affair', 'lodged'), ('lodged', 'behalf'), ('behalf', 'client'), ('client', 'company'), ('company', 'formal'), ('formal', 'notification'), ('notification', 'crime'), ('crime', 'following'), ('following', 'statement'), ('statement', 'data'), ('data', 'sufficiently'), ('sufficiently', 'justify'), ('justify', 'suspicion'), ('suspicion', 'full'), ('full', 'name'), ('name', 'service'), ('service', 'bribe'), ('bribe', 'taking'), ('taking', 'mean'), ('mean', 'bribe'), ('bribe', 'unknown'), ('unknown', 'probably'), ('probably', 'employee'), ('employee', 'company'), ('company', 'name'), ('name', 'code'), ('code', 'document'), ('document', 'question'), ('question', 'connection'), ('connection', 'investigation'), ('investigation', 'forging'), ('forging', 'contract'), ('contract', 'detriment'), ('detriment', 'client'), ('client', 'ongoing'), 

In [66]:
combined_tuples = []
final_bigram_list = []
for bigrams in bigram_list:
  combined_tuples = []
  for tuples in bigrams:
    combined = '_'.join(tuples)
    combined_tuples.append(combined)
  final_bigram_list.append(combined_tuples)
print(final_bigram_list[1])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['born_serving', 'serving_life', 'life_prison', 'prison_prison', 'prison_governor', 'governor_prison', 'prison_access', 'access_version', 'version_state', 'state_gazette', 'gazette_available', 'available_governor', 'governor_request', 'request_subsequent', 'subsequent_chamber', 'chamber_respect', 'respect_legislation', 'legislation_noted', 'noted_available', 'available_paper', 'paper_version', 'version_considered', 'considered_access', 'access_paper', 'paper_version', 'version_sufficient', 'sufficient_found', 'found_prison', 'prison_refusal', 'refusal_grant', 'grant_access', 'access_version', 'version_lawful', 'lawful_however', 'however_noted', 'noted_primary', 'primary_official', 'official_version', 'version_version', 'version_since', 'since_five', 'five_control', 'control_copy', 'copy_edition', 'edition_printed', 'printed_despite', 'despite_prison', 'prison_duty', 'duty_ensure', 'ensure_reasonable', 'reasonable_possibility', 'possibility_searching', 'searching_furthermore', 'furtherm

In [29]:
print(data_bigrams[0])

['life', 'warsaw', 'warsaw', 'polish', 'agent', 'ministry', 'foreign_affair', 'lodged', 'behalf', 'client', 'company', 'formal', 'notification', 'crime', 'following', 'statement', 'data', 'sufficiently', 'justify', 'suspicion', 'full', 'name', 'service', 'bribe', 'taking', 'mean', 'bribe', 'unknown', 'probably', 'employee', 'company', 'name', 'code', 'document', 'question', 'connection', 'investigation', 'forging', 'contract', 'detriment', 'client', 'ongoing', 'time', 'particular', 'request', 'course', 'investigation', 'promptly', 'shortcoming', 'overall', 'investigation', 'question', 'prematurely', 'unjustifiably', 'ground', 'insufficient', 'justify', 'suspicion', 'crime', 'expert', 'report', 'view', 'explanation', 'irrational', 'discontinue', 'investigation', 'charge', 'investigation', 'question', 'notification', 'contain', 'detail', 'regarding', 'allegedly', 'bribe', 'name', 'company', 'supposedly', 'employed', 'warsaw', 'district', 'open', 'inquiry', 'reasoning', 'stated', 'single'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [67]:
id2word = corpora.Dictionary(final_bigram_list)

texts = final_bigram_list

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

word_frequency = []
for doc in corpus:
    # Convert the document to TF-IDF representation
    tfidf_vector = tfidf[doc]
    
    # Iterate over the TF-IDF values for each word in the document
    for word_id, tfidf_score in tfidf_vector:
        # Get the word corresponding to the word ID
        word = id2word[word_id]
        
        word_frequency.append([word,tfidf_score])

df_freq = pd.DataFrame(word_frequency, columns = ['Word', 'Frequency'])
df_sorted = df_freq.sort_values('Frequency', ascending=False)
df_sorted

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word,Frequency
121383,local_religious,0.767142
56443,interim_injunction,0.707488
528705,house_arrest,0.688058
248336,search_seizure,0.566264
284743,ballot_paper,0.563611
...,...,...
530417,notified_writing,0.000002
115591,writing_pursuant,0.000002
115528,pursuant_rule,0.000002
115505,notified_writing,0.000002


In [26]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

word_frequency = []
for doc in corpus:
    # Convert the document to TF-IDF representation
    tfidf_vector = tfidf[doc]
    
    # Iterate over the TF-IDF values for each word in the document
    for word_id, tfidf_score in tfidf_vector:
        # Get the word corresponding to the word ID
        word = id2word[word_id]
        
        word_frequency.append([word,tfidf_score])

df_freq = pd.DataFrame(word_frequency, columns = ['Word', 'Frequency'])
df_sorted = df_freq.sort_values('Frequency', ascending=False)
df_sorted

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word,Frequency
175111,jury,0.933306
39141,blocking,0.903175
15715,interim_injunction,0.901958
73375,reception,0.896688
23490,sister,0.894045
...,...,...
54043,restriction,0.000013
180175,idea,0.000011
33861,everyone,0.000010
174255,everyone,0.000009


In [ ]:
#df_sorted.to_csv("violation_tfidf.csv")
#files.download("violation_tfidf.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
corpus, id2word = tm.tf_idf(id2word, texts, corpus, tfidf)
lda_model = tm.perform_lda(corpus, id2word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10     0.055446 -0.012292       1        1  6.742468
14     0.072073  0.015685       2        1  6.621833
0     -0.063325  0.008953       3        1  6.484582
16    -0.076295 -0.041293       4        1  6.333744
20     0.019660 -0.074293       5        1  6.127581
25    -0.009457 -0.046823       6        1  5.938371
6     -0.034038 -0.028138       7        1  5.210828
3     -0.001553  0.048267       8        1  4.880983
21     0.017873  0.147589       9        1  4.769748
24     0.035145 -0.036071      10        1  4.247504
19     0.022269 -0.010866      11        1  4.198673
7     -0.022967  0.059410      12        1  4.024346
5      0.029091  0.013804      13        1  3.883899
8     -0.020582  0.009281      14        1  3.660112
28     0.014959  0.023764      15        1  3.287786
18    -0.038044  0.023056      16        1  2.660625
12    -0.010947 -0.003950      17        1  2.366267
29    -0.006513  0.001434      18        1  2.230278
22     0.004827 -0.002354      19        1  1.942987
9     -0.008635 -0.020017      20        1  1.891319
13     0.007527 -0.007572      21        1  1.821225
4      0.002750  0.004872      22        1  1.779978
23     0.010973  0.003559      23        1  1.684360
11    -0.003866 -0.005147      24        1  1.677101
26     0.013454 -0.036307      25        1  1.340269
27     0.000550 -0.012904      26        1  1.305880
1     -0.009743 -0.011671      27        1  1.077419
15     0.000677 -0.004917      28        1  0.881491
2     -0.001121 -0.001055      29        1  0.698243
17    -0.000190 -0.004001      30        1  0.230099, topic_info=                           Term         Freq        Total Category  logprob  \
136268       communication_data  1097.000000  1097.000000  Default  30.0000   
33048    telephone_conversation   208.000000   208.000000  Default  29.0000   
11227   necessary_proportionate   236.000000   236.000000  Default  28.0000   
32475             serious_crime   186.000000   186.000000  Default  27.0000   
37209     communication_service   154.000000   154.000000  Default  26.0000   
...                         ...          ...          ...      ...      ...   
56181           statutory_basis     0.019024     0.270816  Topic30  -9.3025   
62891            contact_detail     0.019024     0.270816  Topic30  -9.3025   
2576     information_technology     0.083720     1.773182  Topic30  -7.8208   
62872         child_pornography     0.019025     1.483374  Topic30  -9.3025   
2572        information_society     0.019024     1.502920  Topic30  -9.3025   

        loglift  
136268  30.0000  
33048   29.0000  
11227   28.0000  
32475   27.0000  
37209   26.0000  
...         ...  
56181    3.4187  
62891    3.4187  
2576     3.0214  
62872    1.7181  
2572     1.7050  

[1137 rows x 6 columns], token_table=        Topic      Freq                Term
term                                       
14867      18  0.966993     abortion_doctor
155998      1  0.922577     academic_senate
55057      14  0.578299      access_address
55058      14  0.845159      access_blocked
120718     14  1.099478     access_building
...       ...       ...                 ...
56298      14  0.676321  wholesale_blocking
46758       8  1.003555       without_delay
129641      7  1.021209         work_expert
33687       3  0.839524       working_group
6194       11  0.891765   year_imprisonment

[947 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 15, 1, 17, 21, 26, 7, 4, 22, 25, 20, 8, 6, 9, 29, 19, 13, 30, 23, 10, 14, 5, 24, 12, 27, 28, 2, 16, 3, 18])

In [19]:
print(vis.sorted_terms(topic=14, _lambda=1))

                     Term        Freq       Total Category  logprob  loglift  \
192              district  677.629753  833.039372  Topic14  -2.0061   3.1102   
2486               region  389.172680  711.512616  Topic14  -2.5607   2.7133   
161            defamation  380.200530  703.993378  Topic14  -2.5840   2.7006   
1761                 city  258.776576  378.291493  Topic14  -2.9687   2.9370   
3356           federation  158.633802  228.051572  Topic14  -3.4581   2.9537   
7155                 trip  126.037643  127.046762  Topic14  -3.6881   3.3087   
1755                child  106.625865  191.102933  Topic14  -3.8554   2.7332   
3577                 town  105.207650  234.765579  Topic14  -3.8688   2.5141   
626             statement  104.447160  838.306888  Topic14  -3.8760   1.2340   
202              economic   96.944204  104.349097  Topic14  -3.9506   3.2431   
5093              suspend   87.030062   94.935363  Topic14  -4.0584   3.2298   
239                expert   83.636994  4

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from google.colab import files
import io
uploaded = files.upload()
non_violation_tm = pd.read_csv(io.BytesIO(uploaded['non_violation_facts_processed.csv']))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving non_violation_facts_processed.csv to non_violation_facts_processed.csv


In [ ]:
facts = non_violation_tm['Facts'].tolist()
tm_nn = topic_modeling(facts)
facts_words_nn = tm_nn.organise()
#print(len(facts_words))

#data_bigram, bigram_phases, bigram = tm.bi_grams(facts_words)
#data_bigram_trigram = tm.tri_grams(facts_words, data_bigram, bigram_phases, bigram)

bigram_phrases = gensim.models.Phrases(facts_words_nn, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[facts_words_nn], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(facts_words_nn)
data_bigrams_trigrams = make_trigrams(data_bigrams)

corpus_nn, id2word_nn = tm_nn.tf_idf(data_bigrams_trigrams)
lda_model_nn = tm_nn.perform_lda(corpus_nn, id2word_nn)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
pyLDAvis.enable_notebook()
vis_nn = pyLDAvis.gensim.prepare(lda_model_nn, corpus_nn, id2word_nn, mds="mmds", R=30)
vis_nn

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.067971  0.254623       1        1  14.105345
7      0.043813 -0.313270       2        1  13.351091
8     -0.266895 -0.098658       3        1  12.437003
6     -0.261787  0.167148       4        1  11.833043
9      0.127286 -0.151540       5        1  11.199985
5      0.278773 -0.042535       6        1  10.097928
2      0.172743  0.265216       7        1   9.715848
3     -0.115705 -0.183591       8        1   9.495681
1     -0.058958  0.029303       9        1   3.992097
0      0.148701  0.073304      10        1   3.771978, topic_info=             Term        Freq       Total Category  logprob  loglift
257   information  481.000000  481.000000  Default  30.0000  30.0000
1166         data  197.000000  197.000000  Default  29.0000  29.0000
987        access  121.000000  121.000000  Default  28.0000  28.0000
2464      patient   62.000000   62.000000  Default  27.0000  27.0000
573     statement   80.000000   80.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
843       medical    6.748750   60.121990  Topic10  -5.2357   1.0906
4945   penal_code    5.610233   29.246077  Topic10  -5.4205   1.6264
837        letter    6.201356  102.984522  Topic10  -5.3203   0.4678
1859     district    5.069206   70.514285  Topic10  -5.5219   0.6449
135    defamation    5.064931   60.785815  Topic10  -5.5228   0.7926

[590 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
675       5  0.628159            abortion
675       7  0.048320            abortion
675       8  0.289920            abortion
676       5  0.765670  abortion_performed
676       8  0.170149  abortion_performed
...     ...       ...                 ...
1812      2  0.278499                work
1812      3  0.318284                work
1812      6  0.119357                work
1812     10  0.278499                work
5845      5  0.887415                łódź

[959 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 9, 7, 10, 6, 3, 4, 2, 1])